In [3]:
!pip install -r ../requirements.txt
!pip install sklearn
!pip install tensorflow
!pip install nltk
!pip install pandas
!pip install tensorflow-addons
# !pip install tensorflow addons

  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)


In [6]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/studio-lab-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.metrics import classification_report
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline
from collections import Counter
import  tensorflow_addons as tfa

MAX_LENGTH = 200# Define the maximum number of words to tokenize (DistilBERT can tokenize up to 512)

2022-05-09 15:34:28.354710: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-09 15:34:28.354741: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Create the model

In [2]:
tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa",from_tf=True,)
model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa",)

2022-05-09 15:34:36.267797: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-09 15:34:36.267836: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-09 15:34:36.267903: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default): /proc/driver/nvidia/version does not exist
2022-05-09 15:34:36.269568: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers

# Load and transform the data

In [3]:
datasets = pd.read_csv('../datasets/train_preprocessed.csv').dropna()
X_train,y_train = datasets["content"].astype("string"),datasets["sentiment"].astype("category").cat.codes

In [4]:
# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):
    """""""""
    A function that encodes a batch of texts and returns the texts'
    corresponding encodings and attention masks that are ready to be fed 
    into a pre-trained transformer model.
    
    Input:
        - tokenizer:   Tokenizer object from the PreTrainedTokenizer Class
        - texts:       List of strings where each string represents a text
        - batch_size:  Integer controlling number of texts in a batch
        - max_length:  Integer controlling max number of words to tokenize in a given text
    Output:
        - input_ids:       sequence of texts encoded as a tf.Tensor object
        - attention_mask:  the texts' attention mask encoded as a tf.Tensor object
    """""""""
    
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='max_length', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    
    
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.tolist())

# Define F1 loss

In [5]:
def macro_double_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.
    This version uses the computation of soft-F1 for both positive and negative class for each label.
    
    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        
    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    tn = tf.reduce_sum((1 - y_hat) * (1 - y), axis=0)
    soft_f1_class1 = 2*tp / (2*tp + fn + fp + 1e-16)
    soft_f1_class0 = 2*tn / (2*tn + fn + fp + 1e-16)
    cost_class1 = 1 - soft_f1_class1 # reduce 1 - soft-f1_class1 in order to increase soft-f1 on class 1
    cost_class0 = 1 - soft_f1_class0 # reduce 1 - soft-f1_class0 in order to increase soft-f1 on class 0
    cost = 0.5 * (cost_class1 + cost_class0) # take into account both class 1 and class 0
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

# Create and compile the model

In [5]:
input_ids_layer = tf.keras.layers.Input(shape=(200,), 
                                        name='input_ids', 
                                        dtype='int32')
input_attention_layer = tf.keras.layers.Input(shape=(200,), 
                                              name='input_attention', 
                                              dtype='int32')
last_hidden_state = model.roberta(input_ids_layer, input_attention_layer)[0]#in list

one_more = tf.keras.layers.AveragePooling1D(1,200)(last_hidden_state)
pooling = tf.keras.layers.Reshape((768,))(one_more)
output = tf.keras.layers.Dense(13,activation='softmax')(pooling)
roberta = tf.keras.Model([input_ids_layer, input_attention_layer], output)

In [6]:
params = {'optimizer':tf.keras.optimizers.Adam(learning_rate=1e-4),
          'loss':tf.keras.losses.CategoricalCrossentropy(),#tf.keras.losses.CategoricalCrossentropy()
          "metrics":['accuracy']# tf.keras.metrics.top_k_categorical_accuracy(k=3),metric is ["accuracy"]
}#tfa.metrics.F1Score(num_classes=13,average='macro',threshold = 0.5)
roberta.compile(**params)

In [7]:
roberta.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 200)]        0           []                               
                                                                                                  
 input_attention (InputLayer)   [(None, 200)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124055040   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'input_attention[0][0]']        
                                tentions(last_hidde                                               
                                n_state=(None, 200,                                           

# Created weights for each class and Train

In [9]:
# counts = Counter(y_train)
# weights = {i:1/j for i,j in counts.items()}

In [8]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath="./checkpoints/augmented/",save_best_only=True,save_weights_only=False)
]

roberta.fit(x=[X_train_ids,X_train_attention],
            y=pd.get_dummies(y_train).to_numpy(),
            batch_size=32,
            epochs = 2,
            workers = -1,
            validation_split=0.2,
            callbacks = callbacks#,
            #class_weight = weights#
           )

Epoch 1/2
748/748 [==============================] - ETA: 0s - loss: 2.0361 - accuracy: 0.3177 

2022-05-09 20:58:37.003579: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./checkpoints/augmented/assets


INFO:tensorflow:Assets written to: ./checkpoints/augmented/assets


748/748 [==============================] - 19430s 26s/step - loss: 2.0361 - accuracy: 0.3177 - val_loss: 1.9300 - val_accuracy: 0.3483
Epoch 2/2
748/748 [==============================] - 19214s 26s/step - loss: 1.8829 - accuracy: 0.3692 - val_loss: 1.9867 - val_accuracy: 0.3269


In [9]:
roberta.save_weights('./weights/augmented/saved_weights.h5')

In [10]:
tf.keras.models.save_model(roberta,'./full_models/augmented')

INFO:tensorflow:Assets written to: ./full_models/augmented/assets


INFO:tensorflow:Assets written to: ./full_models/augmented/assets


# Metrics testing part

In [9]:
#load the test set
datasets = pd.read_csv('../datasets/test_preprocessed.csv').dropna()# recreate for non-augmented
X_test,y_test = datasets["content"].astype("string"),datasets["sentiment"].astype("category").cat.codes

X_test_ids, X_test_attention = batch_encode(tokenizer, X_test.tolist())

In [ ]:
#classification report for 2 epochs
from sklearn.metrics import classification_report


y_pred = tf.argmax(roberta.predict([X_test_ids, X_test_attention]),axis = 1)
print(classification_report(y_test, y_pred))

In [ ]:
#EmoRoBERTa(2 epochs) with/without class_weights,f1 loss, augmentation,more epochs
#LSTM with/without class_weights,f1 loss, augmentation,more epochs,2 layers with x neurons
#save each model and calculate f1 for each 